In [9]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [28]:
# basemodels/fixture_data_handler.py

import http.client
import json
import ssl
from datetime import datetime
from django.db import IntegrityError, transaction
from basemodels.models import FikstureModelData, FikstureDataError
from django.conf import settings

class FixtureDataHandler:

    DEFAULT_DATE = datetime.strptime("1900-01-01", "%Y-%m-%d").date()

    def __init__(self, date):
        self.date = date
        self.data_id = date.replace("-", "")
    

    def fetch_fixtures_from_api(self):
        """
        API'den fikstür verilerini çeker, 'events' anahtarını kontrol eder.
        Hata durumunda log_error ile hata kaydı yapar ve boş liste döner.
        """
        try:
            context = ssl._create_unverified_context()
            
            conn = http.client.HTTPSConnection('www.sofascore.com', context=context)
            conn.request('GET', f'/api/v1/sport/football/scheduled-events/{self.date}')
            response = conn.getresponse()

            if response.status == 401:
                self.log_error("AUTH_ERROR", "API Anahtarı Eksik veya Geçersiz")
                return []

            if response.status != 200:
                self.log_error("API_ERROR", f"API isteği başarısız oldu. HTTP Status: {response.status}")
                return []

            data = json.loads(response.read())

            # 'events' anahtarını kontrol et
            if "events" not in data:
                self.log_error("DATA_STRUCTURE_ERROR", "'events' anahtarı API yanıtında bulunamadı")
                return []

            events = data["events"]
            count = len(events)

            return [{
                'data_id': self.data_id,
                'tarih': self.date,
                'data': events,
                'count': count,
                'isprogress': False
            }]

        except KeyError as e:
            self.log_error("DATA_STRUCTURE_ERROR", f"Expected key missing: {e}")
            return []  # Hata durumunda boş liste döner

        except json.JSONDecodeError as e:
            self.log_error("JSON_DECODE_ERROR", str(e))
            return []

        except ConnectionResetError as e:
            self.log_error("CONNECTION_RESET_ERROR", str(e))
            return []

        except Exception as e:
            self.log_error("API_ERROR", str(e))
            return []

    def prepare_data_for_db(self, data):
        """
        API'den gelen veriyi veritabanı için uygun formata dönüştürür.
        """
        return [
            FikstureModelData(
                data_id=self.data_id,
                tarih=self.parse_date(self.date),
                data=item,
                count=item.get('count', 0),
                isprogress=False
            ) for item in data
        ]

    def parse_date(self, date_str):
        """
        Tarih formatını doğrular. Hatalıysa varsayılan tarihi döner.
        """
        try:
            return datetime.strptime(date_str, "%Y-%m-%d").date()
        except ValueError:
            self.log_error("DATA_STRUCTURE_ERROR", f"Geçersiz tarih formatı: {date_str}, varsayılan tarih atanıyor.")
            return self.DEFAULT_DATE

    def save_fixtures_to_db(self):
        """
        API verilerini önce silerek veritabanına kaydeder.
        """
        data = self.fetch_fixtures_from_api()
        records = self.prepare_data_for_db(data)

        if data or data[0]['count'] or  data['count'] < 3:
            # Veri gelmediğini kabul ediyoruz
            self.log_error("DEFAULT_DATA_ERROR", "Veri gelmediği için varsayılan veri kaydedildi.")
            self._save_default_data()
            return

        if records:
            try:
                with transaction.atomic():
                    # Önce aynı data_id'ye sahip veriyi sil
                    FikstureModelData.objects.filter(data_id=self.data_id).delete()
                    # Yeni kayıtları toplu olarak ekle
                    FikstureModelData.objects.bulk_create(records)
                    self.clear_error()  # Hata varsa sil
            except IntegrityError as e:
                self.log_error("INTEGRITY_ERROR", str(e))
        else:
            # Veri boşsa, hata durumunda varsayılan kayıt ekle
            self._save_default_data()

    def _save_default_data(self):
        """
        Hata veya boş veri durumunda varsayılan değerlerle kayıt ekler.
        """
        try:
            with transaction.atomic():
                # Var olan '19000101' data_id'sini sil
                FikstureModelData.objects.filter(data_id='19000101').delete()
                # Yeni varsayılan kayıt oluştur
                FikstureModelData.objects.create(
                    data_id='19000101',
                    tarih=self.DEFAULT_DATE,
                    data={},
                    count=0,
                    isprogress=False
                )
                self.log_error("DEFAULT_DATA_ERROR", "Boş veriyle kayıt yapıldı.")
        except IntegrityError as e:
            self.log_error("INTEGRITY_ERROR", str(e))

    def log_error(self, error_type, message):
        """
        Hata durumunda FikstureDataError tablosuna kayıt ekler.
        """
        FikstureDataError.objects.update_or_create(
            data_id=self.data_id,
            defaults={
                'tarih': self.parse_date(self.date),
                'error_type': error_type,
                'error_message': message
            }
        )

    def clear_error(self):
        """
        Başarılı veri eklemeden sonra ilgili hatayı siler.
        """
        FikstureDataError.objects.filter(data_id=self.data_id).delete()



In [29]:

# Örnek bir tarih ile test
handler = FixtureDataHandler('2024-11-01')
handler.save_fixtures_to_db()
print("İşlem Tamamlandı")


İşlem Tamamlandı
